In [2]:
!pip install transformers   #version: transformers-4.9.2
print("\n \n \n")
!pip install pytorch-lightning  #version: pytorch-lightning-1.4.1

     |████████████████████████████████| 2.6 MB 7.5 MB/s 
     |████████████████████████████████| 636 kB 68.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.3 MB/s 
     |████████████████████████████████| 895 kB 67.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

 
 

     |████████████████████████████████| 916 kB 7.1 MB/s 
     |████████████████████████████████| 829 kB 24.0 MB/s 
     |████████████████████████████████| 118 kB 19.7 MB/s 
     |████████████████████████████████| 272 kB 23.7 MB/s 
     |████████████████████████████████| 1.3 MB 44.6 MB/s 
     |████████████████████████████████| 294 kB 55.9 MB/s 
     |████████████████████████████████| 142 kB 53.6 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=236d43401cfa173b61208f3c5839e01a4577d77d86ec97e615ca77879a540160
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4

In [3]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import random

from tqdm.notebook import tqdm

import re

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as WN

#to read the xml file
import xml.etree.ElementTree as ET

#to store the preprocessed data from the xml
import csv

from transformers import BertConfig, BertModel, BertPreTrainedModel, BertTokenizer
import pytorch_lightning as p_light
from typing import Callable, Optional, Union, List, Dict, Tuple

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# PARAMETERS
bert_model_name = 'bert-base-uncased'
# other possible bert models :  ('bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased',
# 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-base-multilingual-cased')

my_dropout = 0.1   # 0.1 === default dropout of the  'BertTokenizer.from_pretrained'
hidden_size = 768  # 768 === default hidden_size of the  'BertTokenizer.from_pretrained'

my_batch_size = 4  #8 sulla repo forse?????????

In [10]:
class Bert_for_WSD(BertPreTrainedModel):
  def __init__(self, config):
    super().__init__(config)
    self.Bert = BertModel(config)
    self.dropout = nn.Dropout(my_dropout)
    self.add_output_layer = nn.Linear(hidden_size, 1)

    self.init_weights()

In [12]:
bert_config = BertConfig.from_pretrained(bert_model_name, num_labels=2)
bert_model = Bert_for_WSD.from_pretrained(bert_model_name, config=bert_config)
#bert_model = get_bert_model(bert_model_name)

bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)
if '[TGT]' not in bert_tokenizer.additional_special_tokens:
  bert_tokenizer.add_special_tokens({'additional_special_tokens': ['[TGT]']})
bert_model.resize_token_embeddings(len(bert_tokenizer))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing Bert_for_WSD: ['bert.encoder.layer.2.attention.self.value.weight', 'bert.encoder.layer.2.attention.output.dense.bias', 'bert.encoder.layer.4.intermediate.dense.bias', 'bert.encoder.layer.10.attention.self.key.weight', 'bert.encoder.layer.9.attention.self.query.bias', 'bert.encoder.layer.5.attention.self.value.weight', 'bert.encoder.layer.4.attention.output.LayerNorm.weight', 'bert.encoder.layer.11.attention.output.dense.weight', 'bert.encoder.layer.1.attention.self.key.bias', 'bert.encoder.layer.11.attention.self.value.weight', 'bert.encoder.layer.1.attention.self.value.weight', 'bert.encoder.layer.2.output.dense.weight', 'cls.seq_relationship.weight', 'bert.encoder.layer.8.attention.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.self.key.bias', 'bert.encoder.layer.4.attention.self.value.weight', 'bert.encoder.layer.5.attention.self.key.bias', 'bert.encoder.layer.6.attention.self.key.weig

NotImplementedError: ignored